# Process the UN medium-fertility projection dataset
This notebook was used to extract the data I want from the UN projections. I downloaded the excel sheet from [https://population.un.org/wpp/Download/Standard/MostUsed/], then exported the second sheet as a csv. This notebook does the remainder of the processing to spit out the file `population_projections.csv`

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os

output_filepath = os.path.join("..", "data", "population_projections.csv")

# make sure the output directory exists
output_dir = os.path.dirname(output_filepath)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

raw_projections_filepath = os.path.join("..", "raw_data", "un_medium_projections_raw.csv")

df = pd.read_csv(raw_projections_filepath, skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], index_col=0, low_memory=False)
df

In [ ]:
df.columns

In [ ]:
df_filtered = df[["Region, subregion, country or area *", "Type", "Year", "Total Population, as of 1 January (thousands)"]]
df_filtered

In [ ]:
df_filtered["Type"].value_counts()

In [ ]:
# only keep Country/Area
df_filtered = df_filtered[df_filtered["Type"] == "Country/Area"]
df_filtered

In [ ]:
df_filtered = df_filtered.drop(columns=["Type"])
df_filtered

In [ ]:
# Cast Year to int
df_filtered["Year"] = df_filtered["Year"].astype(int)

# Turn the country, year columns into a multiindex
df_filtered = df_filtered.set_index(["Region, subregion, country or area *", "Year"])
df_filtered.index.names = ["Country", "Year"]

# Rename the population column
df_filtered = df_filtered.rename(columns={"Total Population, as of 1 January (thousands)": "Population"})


In [ ]:

# Strip whitespace from the population column
df_filtered["Population"] = df_filtered["Population"].str.strip()

# Remove spaces within numbers in the population column
df_filtered["Population"] = df_filtered["Population"].str.replace(" ", "")

# Parse the population column as an int
df_filtered["Population"] = df_filtered["Population"].astype(int)

# Multiply the population column by 1000 to get the actual population
df_filtered["Population"] = df_filtered["Population"] * 1000

df_filtered

In [ ]:
# Change a few country names to match ethnologue
pop_proj_rename_map = {"Bolivia (Plurinational State of)": "Bolivia", "Brunei Darussalam": "Brunei", "Cabo Verde": "Cape Verde Islands", "China, Hong Kong SAR": "Hong Kong", "China, Macao SAR": "Macao", "Curaçao": "Curacao", "Côte d'Ivoire": "Côte d’Ivoire", "Timor-Leste": "East Timor", "Iran (Islamic Republic of)": "Iran", "Lao People's Democratic Republic": "Laos", "Republic of Moldova": "Moldova", "State of Palestine": "Palestine", "Saint Martin (French part)": "Saint Martin", "Sint Maarten (Dutch part)": "Sint Maarten", "Republic of Korea": "South Korea", "Syrian Arab Republic": "Syria", "Sao Tome and Principe": "São Tomé e Príncipe", "United Republic of Tanzania": "Tanzania", "Türkiye": "Turkey", "United States Virgin Islands": "U.S. Virgin Islands", "United States of America": "United States", "Venezuela (Bolivarian Republic of)": "Venezuela", "Viet Nam": "Vietnam", "China, Taiwan Province of China": "Taiwan", "Wallis and Futuna Islands": "Wallis and Futuna"}

df_filtered.index = df_filtered.index.map(lambda x: (pop_proj_rename_map[x[0]],x[1]) if x[0] in pop_proj_rename_map else x)

df_filtered.loc["Taiwan"]

In [ ]:
# Save the cleaned data
df_filtered.to_csv(output_filepath)